# Class project
This is a class project for students of FORE School of Management. All the students are expected to login on Kaggle during the class, understand data and its features and then ask themselves questions to discover possible relationships among features.  They draw appropriate graphs to bring out such relationships.This is their first Data Visualization project on Kaggle.

In [ ]:
# Ref: https://www.kaggle.com/sakshigoyal7/credit-card-customers
#      https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python
#      https://www.statsmodels.org/stable/contingency_tables.html  
#   

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import graphics libraries
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns

# Plotly libraries
# https://plotly.com/python/plotly-express/#distributions
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.express as px


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Display outputs of multiple commands from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


<a id="Section1"></a>
## Explore Data

In [ ]:
# Read data
df = pd.read_csv("/kaggle/input/credit-card-customers/BankChurners.csv")
df.head(3)

Let us drop Ist column as we will not need 'CLIENTNUM' and also the last two columns. These are not part of collected data.

In [ ]:
# Dropping column with index 0 and the last two columns
df = df.iloc[:, 1:-2]
df.head(3)

In [ ]:
# For conveience let us convert all column names to smallcase:
df.columns = df.columns.str.lower() 
df.columns


In [ ]:
# For convenience let us rename some long column names
#  Copy result of above output here and amend the names

df.columns = [ 'a_flag', 'age', 'gender',
       'dep_count', 'education', 'married',
       'income', 'card_cat', 'months_on_book',
       'total_relationship_count', 'months_inactive_12_mon',
       'contacts_count_12_mon', 'credit_limit', 'total_revolving_bal',
       'avg_open_to_buy', 'total_amt_chng_q4_q1', 'total_trans_amt',
       'total_trans_ct', 'total_ct_chng_q4_q1', 'avg_utilization_ratio']
df.columns

<a id="section-two"></a>
# Questions on Relationships
Looking at the features, here is a set of sample questions. We answer a few of them.

1. Is there a relationship between age vs attrition. Are older persons more loyal?
2. Do ladies churn less and men more or vice-versa 
3. Is there a relationship between income_category vs attrition. Maybe well-off persons churn more rapidly.
4. Do people with higher Eduation level have higher income 
5. Do persons with higher income have higher avg card utilization 
6. What about age wise avg-card-utilization
7. What is gender distribution in the bank
8. Which of the features have good reltaionship with Attrition and which features are important?
9. How is credit_limit related to various features
10. What about credit-card type? 


# Relationships

## Attrition vs gender

Create a contingency or crosstab table. Keep 'target' or 'dependent' variable as *columns* and independent variable as *index*

In [ ]:
# Pandas crosstab function 
ct = pd.crosstab(
                 index = df['gender'],      # Rows
                 columns = df['a_flag'],    # Columns
                 normalize = 'index'        # Row wise proportion
                )
ct

Slightly more attrition on the part of females than males. We can draw a moszic plot to display this.

In [ ]:
# Draw Mosaic plot
# In the index list, keep 'a_flag' first
# https://www.statsmodels.org/stable/generated/statsmodels.graphics.mosaicplot.mosaic.html
_=mosaic(df, index = ['a_flag', 'gender'])

Both seaborn and plotly express can be used to draw heatmaps. Choice of colormap becomes important to make distinctions. You can try, default color map or 'cmap = plt.cm.GnBu'. For a tutorial in colormaps, please see this [link](https://matplotlib.org/tutorials/colors/colormaps.html) 

In [ ]:
sns.heatmap(ct,cmap = plt.cm.coolwarm) ; 

In [ ]:
# Heatmap can also be drawn in plotly express
#  However, it is a heatmap not a mosiac plot with 
#  rectangles of difft sizes:

px.density_heatmap(
                   data_frame =df,
                   x = 'gender',
                   y = 'a_flag'
                   )

## Age vs Attrition


In [ ]:
sns.catplot(kind = 'box', x = 'a_flag', y = 'age', data = df);

The cross-table reveals:
*    People above 54 years age are not using credit cards
*    Among those who are churning, relatively younger age people are more than older people


## Income vs Attrition


In [ ]:

pd.crosstab(
            index = df['income'],      # Rows
            columns = df['a_flag'],    # Columns
            normalize = 'index'        # Row wise proportion
           )

The data reveals:
* Persons in the income group 60K--80K are most loyal
* There is high attrition among <40K and 120K+

In [ ]:
# And here is a mosaic plot for the same

# Decide the figure size of our plot
fig = plt.figure(figsize = (10,10))  
ax = fig.add_subplot(111)

_=mosaic(df,
         index = ['a_flag', 'income'],
         ax = ax, 
         statistic = True
        )

## Card category vs Attrition

In [ ]:
pd.crosstab(
            index = df['card_cat'],    # Rows
            columns = df['a_flag'],    # Columns
            normalize = 'index'        # Row wise proportion
           )

## Credit Limit vs avg_utilization_ratio


In [ ]:
# sns relation plot
# https://seaborn.pydata.org/generated/seaborn.relplot.html#seaborn.relplot
sns.relplot( x = 'credit_limit', y = 'avg_utilization_ratio', data = df) ;

The graph shows that as credit limit is increasing, avg_utilization is decreasing exponentially. But what about total transaction amount? Let us see:

## Credit Limit vs Total Trans Amt

In [ ]:
sns.relplot( x = 'credit_limit', y = 'total_trans_amt', data = df ) ;

There dows not appear to be any relationship here between credit limit and total_trans_amt. Does credit limit have to do something with Income? But before that, let us see if there is any relationship between a_flag, credit limit and total_trans_amt. Let us see:


In [ ]:
sns.relplot( x = 'credit_limit', y = 'total_trans_amt', data = df, hue = 'a_flag' ) ;

Yes, there is a pattern. One can confidently say that no one whose *total_trans_amt* is above 12500/- churns. Besides, there is a pattern of those churning for certain bands of *'total_trans_amt'*.<br>
Let us now proceed with discovering relationship between *credit_limit* and *income*.

## Credit Limit vs Income

In [ ]:
# credit limit vs income
sns.catplot(x = 'credit_limit', y = 'income', kind = 'box', data = df);

Yes, definitely a relatioship exists between credit limit and income. What if I want to condition by *'a_flag'*?

In [ ]:
# credit limit vs income vs a_flag
sns.catplot(x = 'credit_limit', y = 'income', kind = 'box', data = df, hue = 'a_flag');

The trend of pattern remains the same. Those with high income have higher credit limit whether they are in churn class or loyal group. Let us now look at the relationship of age vs avg_utilization_ratio

## Age vs Avg Utilization Ratio

In [ ]:
# Does 'age' has a relationship with 'avg_utilization_ratio'
sns.relplot( x = 'age', y = 'avg_utilization_ratio', data = df) ;

Obviouly age has no relationship with avg utilization ratio. It appears somewhat unintutive as those Let us now see the relationship of attrition to avg_utilization ratio.

## Distribution of Avg Utilization Ratio

In [ ]:
# Boxplot of avg_utilization_ratio conditioned by a_flag
sns.catplot(kind = 'box', data = df, x = 'a_flag', y = 'avg_utilization_ratio') ;


Surely attrition is affected by avg_utilization_ratio. We can also look at these through density plots.

In [ ]:
# We can also draw a density plot of these two feature
sns.kdeplot(data=df, x="avg_utilization_ratio", hue="a_flag") ;

In the case of churning customers, avg_utilization ratio peaks around 0.0 while in the case of loyal customers after a peak, curve is somewhat flat till 0.8 when it touches down.

## Pairplots of float features

In [ ]:
# We will draw pairplots. First of float features
# then of integer features. This distiction by us between float and int
# is just for conveience.
df_f = df.select_dtypes(include = ['float64'])
# Add to it attrition flag also
df_f['a_flag'] = df['a_flag']

In [ ]:
# Pairplot now conditioned by attrition
sns.pairplot(df_f, hue = 'a_flag') ;

Several conclusiuons can be drawn:
* i)   *'credit_limit'* and *'avg_open_to_buy'* are strongly related
* ii)  *'credit_limit'* has relationship with *'avg_utilization_ratio'*. *'avg_utilization_ratio'* is exponentially decresing with *'credit_limit'*
* iii) Not exactly, but somewhat linear relatioship exists between *'total_ct_chng_q4_q1'* and *'total_amt_chng_q4_q1'*
* iv)  Both *'total_ct_chng_q4_q1*' and *'total_amt_chng_q4_q1'* show a pattern with respect to *'a_flag'*
*  v)  *'credit_limit'* does not appear to be related to both *'total_ct_chng_q4_q1*' and *'total_amt_chng_q4_q1'* 


## Pairplots of integer features

In [ ]:
# Integer features, next
df_i = df.select_dtypes(include = ['int64'])
# Add 'a_flag' also
df_i['a_flag'] = df['a_flag']

In [ ]:
sns.pairplot(df_i, hue = 'a_flag') ;

In [ ]:
sns.relplot(data = df,x = "credit_limit", y = "avg_utilization_ratio", col = "card_cat", hue = 'a_flag');

# Joint distributions
Joint distributions of two continuous features is a scatter plot of two features with distributions of each of the feature also plotted on the margins. We will  condition such plots by 'a_flag'.

In [ ]:
sns.jointplot(x = 'avg_utilization_ratio', y = 'avg_open_to_buy', data = df, hue ='a_flag') ;

Most attrition remains at the borders of this plot. So a pattern is evident.

Another jointplot between 'total_revolving_bal' and 'total_trans_amt',

In [ ]:
sns.jointplot(x = 'total_revolving_bal', y = 'total_trans_amt', data = df, hue ='a_flag') ;

Here again one can clearly see bands of attrition along X-axis. Let us see one last jointplot between 'total_amt_chng_q4_q1' and 'total_trans_amt' .

In [ ]:
sns.jointplot('total_amt_chng_q4_q1', 'total_trans_amt' , data =df , hue = 'a_flag', alpha = 0.4)

This joint plot shows a clear and very distinguishing pattern of those churning and those not. In fact just these two features should be good enough to make strong prediction of churning customers. Let us see if this is true. We will use a very simple approach of k-nearest neighbour for making prediction.

## A quick classification modeling

In [ ]:
# Call the requisite libraries
from sklearn.ensemble import RandomForestClassifier    # Classifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split   # Data splitter
from sklearn.metrics import f1_score

In [ ]:
# Extract a data frame with just two features + target
dx = df[['total_amt_chng_q4_q1', 'total_trans_amt','a_flag']].copy()
dx.head(3)


In [ ]:
# Separate target and predictors.
y = dx.pop('a_flag')
# Also let us map
y = y.map({'Existing Customer' : 0, 'Attrited Customer' : 1 })
X = dx.values

In [ ]:
ss = StandardScaler()
X_s = ss.fit_transform(X)

In [ ]:
# Split data into train and test

X_train,X_test,y_train,t_test = train_test_split(X_s,y,test_size = 0.3)

In [ ]:
# Instantiate classifier and train it over our two-features data
rnn = RandomForestClassifier(n_estimators= 200)
rnn.fit(X_train,y_train)

In [ ]:
# Make predictions
y_pred = rnn.predict(X_test)
# Accuracy
np.sum(t_test == y_pred)/y_pred.size    # 89%

In [ ]:
# And f1-score
f1_score(t_test, y_pred)    # 0.66

## Joint distribution again

In [ ]:
sns.jointplot('total_amt_chng_q4_q1','total_ct_chng_q4_q1', data = df, hue = 'a_flag');

Correlation of numeric features

In [ ]:
# Ref: https://heartbeat.fritz.ai/seaborn-heatmaps-13-ways-to-customize-correlation-matrix-visualizations-f1c49c816f07
fig = plt.figure(figsize = (15,10))  # 15: width, 10: height
ax = fig.add_subplot(111)
sns.heatmap(df.corr(), annot = True, vmin=-1, vmax=1, center= 0, cmap= 'coolwarm', ax = ax, fmt = '.1g');

The graph displays both -ve and +ve correlations. Clearly you can see that some features are very closely correlated.

Before we close, let us draw one last graph using plotly express to display relationship between education and credit_limt, conditioned by 'a_flag',

In [ ]:
fig = px.box(df, x="education", y="credit_limit", color="a_flag", notched=True) ;
fig.show()

# Discovering structure in data
Does our data possess some overall structure? We will use different plots to discover existence or absence of such structures. And also see what kind of plots we get when we use random data, instead. 

In [ ]:
# Let us process data first
# We select only numeric data 
num_data = df.select_dtypes(include = ['float64', 'int64']).copy()
num_data.head(2)
num_data.shape      # (10127, 14)  
num_data.columns

In [ ]:
#  Here is a full list of numeric columns
#  And our dataset

cols=['age', 'dep_count', 'months_on_book', 'total_relationship_count',
       'months_inactive_12_mon', 'contacts_count_12_mon', 'credit_limit',
       'total_revolving_bal', 'avg_open_to_buy', 'total_amt_chng_q4_q1',
       'total_trans_amt', 'total_trans_ct', 'total_ct_chng_q4_q1',
       'avg_utilization_ratio']

num_data = df[cols]


In [ ]:
# We will scale all numeric data
# Create an instance of StandardScaler object
ss= StandardScaler()

# Use fit and transform method to transform all numeric data
nc = ss.fit_transform(num_data.loc[:,cols])

nc.shape     # (10127, 14)

#     Transform numpy array back to pandas dataframe
#     as we will be using pandas plotting functions
nc = pd.DataFrame(nc, columns = cols)
nc.head(2)

In [ ]:
# Add target column
# And transform it to 1,0

nc['a_flag'] = df['a_flag']
nc['a_flag'] = nc.a_flag.map({"Existing Customer" : 0, "Attrited Customer" : 1})
nc.head(2)

## Parallel co-ordinates plots
See [here](https://en.wikipedia.org/wiki/Parallel_coordinates) for explanations

In [ ]:
# Parallel coordinates with 'nc' data
fig = plt.figure() ;
ax = pd.plotting.parallel_coordinates(nc,
                                      'a_flag',
                                       colormap= plt.cm.winter
                                       ) ;

plt.xticks(rotation=90) ;
plt.title("Parallel chart with credit-card data") ;


## Andrews plots
See [here](https://en.wikipedia.org/wiki/Andrews_plot) for how it is derived.

In [ ]:
# Andrews plots with nc data
fig = plt.figure();
pd.plotting.andrews_curves(nc,
                           'a_flag',
                            colormap = plt.cm.winter
                           );
plt.xticks(rotation=90);
plt.title("Andrews curve with credit-card data");



## Radviz plot
See this [link](https://www.researchgate.net/figure/Radviz-plot-with-two-data-examples-from-the-table-Springs-are-drawn-to-illustrate-how_fig1_225719533) for how it works

In [ ]:
# Radviz plot
# https://pandas.pydata.org/docs/reference/api/pandas.plotting.radviz.html

fig = plt.figure();
pd.plotting.radviz(nc,
                   class_column ='a_flag',
                   colormap= plt.cm.winter,
                   alpha = 0.4
                   );

## Plots with Random data

In [ ]:
# For the sake of compairsion, we also generate random data
# Generate random data
rng = np.random.default_rng()
nc_rand = pd.DataFrame(rng.normal(size = (10127, 14)),
                       columns = cols    # Assign column names, just like that
                       )

nc_rand['a_flag'] = df['a_flag']
nc_rand['a_flag'] = nc_rand.a_flag.map({"Existing Customer" : 0, "Attrited Customer" : 1})
nc_rand.head(2)

In [ ]:
# Parallel coordinates with random data
fig = plt.figure() ;
ax = pd.plotting.parallel_coordinates(nc_rand,
                                      'a_flag',
                                      colormap= plt.cm.winter
                                       ) ;

plt.xticks(rotation=90) ;
plt.title("Parallel chart with random data") ;


In [ ]:
# Andrews plots with random data
fig = plt.figure();
pd.plotting.andrews_curves(nc_rand,
                           'a_flag',
                            colormap = plt.cm.winter
                           );
plt.xticks(rotation=90);
plt.title("Andrews curve with random data");

# t-sne plot
For a very good and simple explanation of how t-sne works, see [this video](https://www.youtube.com/watch?v=NEaUSP4YerM)


In [ ]:
# Use t-sne to look at the structure of data
# (t-distributed Stochastic Neighbor Embedding)

from sklearn.manifold import TSNE

# Project all data but 'a_flag' on two axis
# Also just replace nc with nc_rand and try again

X_embedded = TSNE(n_components=2).fit_transform(nc.iloc[:,:-1])
#X_embedded.shape    # (10127, 2), numpy array
dsne = pd.DataFrame(X_embedded, columns=['X','Y'])

# Clear clusters of 1 and 0 can be seen in the plot
sns.relplot(x = "X",
            y = "Y",
            hue = nc.a_flag,    # Colur each point as per 1 or 0
            data = dsne
            ) ;


# Students Exercises:
Students may perform any one of the following four projects:

1. Predicting Churn for [Bank Customers](https://www.kaggle.com/adammaus/predicting-churn-for-bank-customers)
2. Understand [Housing prices data](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)
3. Predict [Telecom churn](https://www.kaggle.com/blastchar/telco-customer-churn)
4. Analyse [Supermarket sales data](https://www.kaggle.com/aungpyaeap/supermarket-sales) 
      


In [ ]:
###### ####### --XXX -- ###### ####### 